In [17]:
import pickle
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [18]:
tellco_engagement_df = pd.read_csv("../data/user_engagement_data.csv")
tellco_engagement_df.head(5)

,MSISDN/Number,Cluster,number of xDR Sessions,Dur (ms),Total Data Volume (Bytes)
0,33601001722,2,1,116720140.0,8.786906e+08
1,33601001754,0,1,181230963.0,1.568596e+08
2,33601007832,0,1,49878024.0,4.223207e+08
3,33601008617,1,2,37104453.0,1.457411e+09
4,33601010682,1,2,253983077.0,6.152172e+08


In [19]:
tellco_experience_df = pd.read_csv("../data/user_experience_data.csv")
tellco_experience_df.head(5)

,MSISDN/Number,Cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33601001722,0,46.000000,76.0,2.156957e+07
1,33601001754,0,31.000000,99.0,2.156957e+07
2,33601007832,2,84.000000,248.0,7.607247e+05
3,33601008617,1,119.000000,43342.0,3.094040e+07
4,33601010682,1,151.458589,7908.0,2.233199e+07


In [20]:
with open("../models/user_engagement.pkl", "rb") as f:
    kmeans1 = pickle.load(f)

In [21]:
with open("../models/user_experience.pkl", "rb") as f:
    kmeans2 = pickle.load(f)

In [22]:
engagement_df = tellco_engagement_df.set_index('MSISDN/Number')[['number of xDR Sessions', 'Dur (ms)', 'Total Data Volume (Bytes)']]
scaler = StandardScaler()
scale_data = scaler.fit_transform(engagement_df)
scale_data

array([[-0.49203077, -0.16503338,  0.3859319 ],
       [-0.49203077,  0.34330808, -1.09205693],
       [-0.49203077, -0.69174528, -0.54851071],
       ...,
       [-0.49203077, -1.01535345,  0.05003262],
       [-0.49203077,  0.02620111, -0.42977986],
       [-0.49203077,  0.80534306, -0.19230338]])

In [11]:
normalized_data = normalize(scale_data)
normalized_data

array([[-0.76078506, -0.25517698,  0.59673345],
       [-0.39488482,  0.27552575, -0.87644255],
       [-0.48683242, -0.68443693, -0.54271565],
       ...,
       [-0.4356575 , -0.89902171,  0.04430025],
       [-0.75253815,  0.04007338, -0.65732828],
       [-0.51085735,  0.83615791, -0.19966149]])

In [23]:
less_engaged_cluster = 3
distance = kmeans1.fit_transform(normalized_data)
distance_from_less_engagement = list(
    map(lambda x: x[less_engaged_cluster], distance))
tellco_engagement_df['engagement_score'] = distance_from_less_engagement
tellco_engagement_df.head(5)

,MSISDN/Number,Cluster,number of xDR Sessions,Dur (ms),Total Data Volume (Bytes),engagement_score
0,33601001722,2,1,116720140.0,8.786906e+08,0.763547
1,33601001754,0,1,181230963.0,1.568596e+08,1.139935
2,33601007832,0,1,49878024.0,4.223207e+08,1.335330
3,33601008617,1,2,37104453.0,1.457411e+09,1.551139
4,33601010682,1,2,253983077.0,6.152172e+08,1.487249


In [24]:
experience_df = tellco_experience_df.set_index('MSISDN/Number')[['Total Avg RTT (ms)', 'Total Avg Bearer TP (kbps)', 'Total TCP Retrans. Vol (Bytes)']]
scaler = StandardScaler()
scale_data = scaler.fit_transform(experience_df)
scale_data

array([[-0.67078091, -0.55096718,  0.0441717 ],
       [-0.81381178, -0.55023357,  0.0441717 ],
       [-0.30843605, -0.54548107, -1.12899202],
       ...,
       [-0.61356857, -0.22278955, -1.15958728],
       [-0.72799326, -0.54969134,  0.0441717 ],
       [ 0.10595858, -0.5523387 ,  0.0441717 ]])

In [25]:
normalized_data = normalize(scale_data)
normalized_data

array([[-0.77174548, -0.63389763,  0.05082033],
       [-0.82758239, -0.55954415,  0.04491913],
       [-0.23886777, -0.42244688, -0.87434594],
       ...,
       [-0.46108945, -0.16742369, -0.87141599],
       [-0.79711681, -0.60188498,  0.04836584],
       [ 0.18782248, -0.97907712,  0.07829887]])

In [26]:
worst_experience_cluster = 1
distance = kmeans2.fit_transform(normalized_data)
distance_from_worst_experience_cluster = list(
    map(lambda x: x[worst_experience_cluster], distance))
tellco_experience_df['experience_score'] = distance_from_worst_experience_cluster
tellco_experience_df.head()

,MSISDN/Number,Cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),experience_score
0,33601001722,0,46.000000,76.0,2.156957e+07,1.544919
1,33601001754,0,31.000000,99.0,2.156957e+07,1.545474
2,33601007832,2,84.000000,248.0,7.607247e+05,1.589208
3,33601008617,1,119.000000,43342.0,3.094040e+07,0.656053
4,33601010682,1,151.458589,7908.0,2.233199e+07,1.046647


In [ ]:
# user_engagement_df.rename(columns={'Cluster': 'engagement_cluster'}, inplace=True)
# user_engagement_df.head()
# user_experience_df.rename(columns={'cluster': 'experience_cluster'}, inplace=True)
# user_experience_df.head()

In [27]:
user_satisfaction_df = pd.merge(tellco_engagement_df, tellco_experience_df, on='MSISDN/Number')
user_satisfaction_df['satisfaction_score'] = (
    user_satisfaction_df['engagement_score'] + user_satisfaction_df['experience_score'])/2
user_satisfaction_df.head()

,MSISDN/Number,Cluster_x,number of xDR Sessions,Dur (ms),Total Data Volume (Bytes),engagement_score,Cluster_y,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),experience_score,satisfaction_score
0,33601001722,2,1,116720140.0,8.786906e+08,0.763547,0,46.000000,76.0,2.156957e+07,1.544919,1.154233
1,33601001754,0,1,181230963.0,1.568596e+08,1.139935,0,31.000000,99.0,2.156957e+07,1.545474,1.342704
2,33601007832,0,1,49878024.0,4.223207e+08,1.335330,2,84.000000,248.0,7.607247e+05,1.589208,1.462269
3,33601008617,1,2,37104453.0,1.457411e+09,1.551139,1,119.000000,43342.0,3.094040e+07,0.656053,1.103596
4,33601010682,1,2,253983077.0,6.152172e+08,1.487249,1,151.458589,7908.0,2.233199e+07,1.046647,1.266948


In [28]:
user_satisfaction_df = user_satisfaction_df[['MSISDN/Number', 'engagement_score',
                        'experience_score', 'satisfaction_score']]
user_satisfaction_df.set_index('MSISDN/Number', inplace=True)
user_satisfaction_df.head()

,engagement_score,experience_score,satisfaction_score
MSISDN/Number,,,
33601001722,0.763547,1.544919,1.154233
33601001754,1.139935,1.545474,1.342704
33601007832,1.335330,1.589208,1.462269
33601008617,1.551139,0.656053,1.103596
33601010682,1.487249,1.046647,1.266948


In [29]:
user_satisfaction_df.sort_values('satisfaction_score', ascending=False).head(10)

,engagement_score,experience_score,satisfaction_score
MSISDN/Number,,,
33662819510,1.868994,1.586842,1.727918
33615991922,1.834298,1.603974,1.719136
33610116506,1.876117,1.560590,1.718353
33668355657,1.859386,1.573645,1.716516
33761712588,1.871823,1.558289,1.715056
33632638205,1.881886,1.539271,1.710578
33615080072,1.866384,1.554764,1.710574
33676832615,1.868894,1.542333,1.705613
33672026610,1.885475,1.525597,1.705536


In [30]:
X = user_satisfaction_df[['engagement_score', 'experience_score']]
y = user_satisfaction_df[['satisfaction_score']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [31]:
y_pred = model.predict(X_test)

In [33]:
user_satisfaction_df1 = user_satisfaction_df[['engagement_score', 'experience_score']]
user_satisfaction_df1

,engagement_score,experience_score
MSISDN/Number,,
33601001722,0.763547,1.544919
33601001754,1.139935,1.545474
33601007832,1.335330,1.589208
33601008617,1.551139,0.656053
33601010682,1.487249,1.046647
...,...,...
33789960306,0.234576,1.348296
33789967113,1.149889,1.543401
33789996170,1.343077,1.593265


In [34]:
scaler = StandardScaler()
scale_data = scaler.fit_transform(user_satisfaction_df1)
scale_data

array([[-1.33620157,  0.85042177],
       [-0.21412321,  0.85212284],
       [ 0.36838262,  0.98616545],
       ...,
       [ 0.39147823,  0.99860036],
       [-0.86681468,  0.85214471],
       [-1.6838591 ,  0.24887614]])

In [35]:
normalized_data = normalize(scale_data)
normalized_data

array([[-0.84362955,  0.53692568],
       [-0.24370574,  0.96984922],
       [ 0.34993273,  0.93677483],
       ...,
       [ 0.36498267,  0.93101431],
       [-0.71311541,  0.70104665],
       [-0.98925318,  0.14621266]])

In [36]:
kmeans = KMeans(n_clusters = 3, random_state = 42)
y_kmeans = kmeans.fit_predict(normalized_data)
X = np.array(normalized_data)
y_kmeans

array([0, 2, 2, ..., 2, 0, 0])

In [37]:
clustered_tellco_satisfaction_df = user_satisfaction_df.copy()
clustered_tellco_satisfaction_df.insert(0, 'Cluster', y_kmeans)
clustered_tellco_satisfaction_df

,Cluster,engagement_score,experience_score,satisfaction_score
MSISDN/Number,,,,
33601001722,0,0.763547,1.544919,1.154233
33601001754,2,1.139935,1.545474,1.342704
33601007832,2,1.335330,1.589208,1.462269
33601008617,1,1.551139,0.656053,1.103596
33601010682,1,1.487249,1.046647,1.266948
...,...,...,...,...
33789960306,0,0.234576,1.348296,0.791436
33789967113,2,1.149889,1.543401,1.346645
33789996170,2,1.343077,1.593265,1.468171


In [39]:
clustered_tellco_satisfaction_df.groupby('Cluster').agg(
    {'satisfaction_score': 'mean', 'experience_score': 'mean'})

,satisfaction_score,experience_score
Cluster,,
0,1.138640,1.446422
1,1.172989,0.880226
2,1.401160,1.490780


In [ ]:
engine = create_engine('mysql+pymysql://root:@localhost/tellco')

In [ ]:
# writing to database
try:
    print('writing to the database')
    frame = user_satisfaction_df.to_sql(
        "tellco_analysis", con=engine, if_exists='replace')
except Exception as e:
  print("Error writing to database: ", e)
writing to the database
# reading from database
pd.read_sql("select * from tellco.tellco_analysis", engine)

In [ ]:
user_satisfaction_df.to_csv('../data/user_satisfaction_data.csv')